# Game Recommendation Dataset - 2A

### Datasets Information

### Goals:
If your dataset is 3A, you need to make a game  recommendation system
1. [LO 1,2,3 – 8 Points] Perform Exploratory Data Analysis (EDA) on the dataset. Identify the key  findings and anomalies existing in the dataset. Make sure to handle the anomalies accordingly! 
2. [LO 1,2,3 – 7 Points] Build a content-based recommender system. Wrap it as a function that takes a  string input and returns 5 recommended items! 
3. [LO 1,2,3,4 – 5 Points] Evaluate the function using at least 3 inputs, analyze whether the recommended  items are associated with each of the inputs! 
4. [LO 4 – 5 Points] Make a presentation video explaining the key points of your analysis (max. 5  minutes)! 


### Variables

1. Index: row index of the game in the table 
2. Name: the name of the game 
3. Platform: the platform for which the game was made 
4. Year_of_Release: the year in which the game was released 
5. Genre: the genre of the game 
6. Publisher: the game’s publisher 
7. NA_Sales: number of copies sold in North America (millions) 
8. EU_Sales: number of copies sold in Europe (millions) 
9. JP_Sales: number of copies sold in Japan (millions) 
10. Other_Sales: number of copies sold in other countries (millions) 
11. Global_Sales: the total number of copies sold globally (millions) 
12. Critic_Score: the average score given by game critics (0-100) 
13. Critic_Count: the number of game critics that reviewed the game 
14. User_Score: the average score given by users (0-10) 
15. User_Count: the number of users playing the game 
16. Developer: the game’s developer 
17. Rating: the ESBR categorization of the game (such as E=Everyone, T=Teen, M=Mature, etc.) 



### Step 1 : importing library and datasets

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
datasets = pd.read_csv("/Users/aaron/Documents/Semester 3/UAS/ML/3A.tsv",sep='\t')
datasets

,index,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,0,Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,-706.0,NaN,NaN
2,2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,-92.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16923,16923,Aliens: Colonial Marines,PS3,2013,Shooter,Sega,0.20,0.29,0.00,0.11,0.60,43.0,25.0,3.6,295.0,Gearbox Software,M
16924,16924,Backyard Wrestling: Don't Try This at Home,PS2,2003,Fighting,Eidos Interactive,0.30,0.23,0.00,0.08,0.60,51.0,24.0,7.8,29.0,Paradox Development,M
16925,16925,Yakuza: Dead Souls,PS3,2011,Shooter,Sega,0.09,0.06,0.42,0.03,0.60,64.0,55.0,7.4,69.0,Ryu ga Gotoku Studios,M
16926,16926,Fight Night Round 2,XB,2005,Fighting,Electronic Arts,0.42,0.16,0.00,0.02,0.60,88.0,48.0,8.8,27.0,EA Sports,T


### Step 2 : EDA

**a. see the unique value and data type of each columns**

In [3]:
for column in datasets.columns:
    unique_values = datasets[column].nunique()
    print(f"{column}: {unique_values}")

index: 16928
Name: 11562
Platform: 31
Year_of_Release: 49
Genre: 12
Publisher: 581
NA_Sales: 402
EU_Sales: 307
JP_Sales: 244
Other_Sales: 155
Global_Sales: 629
Critic_Score: 82
Critic_Count: 106
User_Score: 96
User_Count: 1888
Developer: 1696
Rating: 8


In [4]:
print(datasets.dtypes)

index                int64
Name                object
Platform            object
Year_of_Release      int64
Genre               object
Publisher           object
NA_Sales           float64
EU_Sales           float64
JP_Sales           float64
Other_Sales        float64
Global_Sales       float64
Critic_Score       float64
Critic_Count       float64
User_Score          object
User_Count         float64
Developer           object
Rating              object
dtype: object


Dari informasi di atas mari kita lihat apa yang dapat kita ambil untuk kolom:
1. Index, for index we're just gonna straight to drop it because it's an identifier.
2. Name, for this column is this most important column because this is the column that well be our output for the recomendation system. So we're gonna see is there any dups and null for it, because it has to be clean before we deploy the recomendattion system.But ofc null removing or transformation need to be applied to this column as well.
3. Platform, platform as well can be a key to the recomender system because not all games available in one platform, this could help the recomender group as well which game is in one platform or cluster so the system can recomnded the same game. 

4. Year or Release, could also be a trens for the system so it will also recomending something that also new for insertion new game and viceversa. This column is clean from missing value so we can straight to see its unique value and analyze it from there.
5. Genre, Genre is also one of the most important column, because it's a column that specify the type of the game. So it would be a huge help from this column. So for next we're gonna see the Missing values from it.

6. Publisher, publisher is also one of the column that can bring a good recomendation. Usually when it's a big company that publish it, it's usually a games that people will enjoy it. This also can help the system recomend you about the another games that the same publisher has made. 

7. NA_Sales          
8. EU_Sales           
9. JP_Sales            
10. Other_Sales        
11. Global_Sales  

for number 7,8 and 9,10,11 im just gonna straight to it cause i feel the three of this column has the same points. It's all about the sales. this is also a good info for our system later, a lot of sales indicates that a lot of people interested to it and this could be a key for our system to recomended a game later.

12. Critic_Score    
critic score is one out of the two crucial column in our table, because i just see through the future that this column has a lot of missing values. Why i said crucial ? Because this is one of the most valuable column that will help our system gave a good recomender by seeing positive reviews of critics from other people. Usually the system will avoid the ones that has low critic score. So in the further work, im gonna drop the rows of the missing values because i know that if we input score's to it, it's not pure anymore and can messed up the recomend system that it'll gave to us.

13. Critic_Count 
critic count it also in sync with critic score, but the different is usually the more people give critic the more system will know it's trustworthy or not. And yeah this has a lot of missing values too so good luck me for cleaning this.

14. User_Score  
users score also the same like critic score one of the most important thing that we're gonna have in our column to see star ratings from other user that have played the game. This could also be used for the machine to double check the critic that has been given to the game. 

15. User_Count
in sync with user score, the the more the user has given a score the more system will know it's trustworthy or not. And yeah this is different it doesnt have any missing values. Hooray.

16. Developer   
will maintain it's games so that people will enjoy it. This also can help the system recomend you about the another games that the same developper has made. Like example in app store usually when we download a games, there are recommendation games was made by the same developper that we downloaded it.  
        
17. Rating     
Last but not least the rating, for rating it's like age category. this is helpful so we can filter the one that will give to it's on the same rating or compatible. This something netflix kids uses probably i guess. That it filter film that above age to recomend to kids as well.

**b. dropping duplicates ( if there are any)**

In [5]:
datasets.duplicated().sum()

0

no duplicates indicates that there's not gonna be a mainstream insight that we gonna get from each data rows.

**c. dropping unuseful column**

Now im gonna delete the columns for Index because it's an identifier column so it gotta be removed.

In [6]:
# drop kolom id 
datasets.drop(["index"], axis="columns", inplace=True)
datasets

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,-706.0,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,-92.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16923,Aliens: Colonial Marines,PS3,2013,Shooter,Sega,0.20,0.29,0.00,0.11,0.60,43.0,25.0,3.6,295.0,Gearbox Software,M
16924,Backyard Wrestling: Don't Try This at Home,PS2,2003,Fighting,Eidos Interactive,0.30,0.23,0.00,0.08,0.60,51.0,24.0,7.8,29.0,Paradox Development,M
16925,Yakuza: Dead Souls,PS3,2011,Shooter,Sega,0.09,0.06,0.42,0.03,0.60,64.0,55.0,7.4,69.0,Ryu ga Gotoku Studios,M
16926,Fight Night Round 2,XB,2005,Fighting,Electronic Arts,0.42,0.16,0.00,0.02,0.60,88.0,48.0,8.8,27.0,EA Sports,T


**d. Examine the value???**

In [7]:
columns_to_check = ['Name','Year_of_Release', 'Genre','Developer','Rating','User_Score']  # Specify the columns you want to analyze

for column in columns_to_check:
    unique_values = datasets[column].unique()
    print(f"Column: {column}")
    print(f"Unique Values ({len(unique_values)}): {unique_values}")
    print("-" * 50)

Column: Name
Unique Values (11563): ['Wii Sports' 'Super Mario Bros.' 'Mario Kart Wii' ...
 'Woody Woodpecker in Crazy Castle 5' 'LMA Manager 2007'
 'Haitaka no Psychedelica']
--------------------------------------------------
Column: Year_of_Release
Unique Values (49): [2006 1985 2008 2009 1996 1989 1984 2005 1999 2007 2010 2013 2004 1990
 1988 2002 2001 2011 1998 2015 2012 2014 1992 1997 1993 1994 1982 2016
 2003 1986 2000    6 1995 1991 1981 1987 1980    8 1983    7    1    4
    2    5   10    3    9 2020 2017]
--------------------------------------------------
Column: Genre
Unique Values (13): ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' nan]
--------------------------------------------------
Column: Developer
Unique Values (1697): ['Nintendo' nan 'Good Science Studio' ... 'Big Red Software'
 'Atomic Games' 'Interchannel-Holon']
--------------------------------------------------
Column: Rating
Uniqu

Year Of Release
first of all for year there is an anomaly because we can see there are years such as 8,7,1,4,2,5,10,and 3 so well first of all i want to input it manually then i realize how many that have the same probelm so for the year cause there are too much if we input one by one so we're just gonna input a threshold too it. We'll gonna change it to Unkown. So we now we'll gonna give them a special attention.

Genre
i've never heard of genre called Misc though i search the google and it give me a name of a band sooo judging from the name i'll change it to Music

for Rating, 
i searched and there arent any K-A Rating, so im guessing K - A it means Kids until Adult so i'll categorized E (everyone) , also judging from the game names it's for kids - adult

In [8]:
datasets['Year_of_Release'] = datasets['Year_of_Release'].apply(lambda x: x if x >= 1980 else 'Unknown')
print(datasets['Year_of_Release'].unique())

[2006 1985 2008 2009 1996 1989 1984 2005 1999 2007 2010 2013 2004 1990
 1988 2002 2001 2011 1998 2015 2012 2014 1992 1997 1993 1994 1982 2016
 2003 1986 2000 'Unknown' 1995 1991 1981 1987 1980 1983 2020 2017]


i guess i'll change misc to music according to games like just dance 3 and 2 and etc

In [9]:
datasets['Genre'] = datasets['Genre'].replace({'Misc': 'Music'})
unique_values = datasets['Genre'].unique()
for value in unique_values:
    print(value)

Sports
Platform
Racing
Role-Playing
Puzzle
Music
Shooter
Simulation
Action
Fighting
Adventure
Strategy
nan


In [10]:
datasets['Rating'] = datasets['Rating'].replace({'K-A': 'Everyone'})
print(datasets['Rating'].unique())

['E' nan 'M' 'T' 'E10+' 'Everyone' 'AO' 'EC' 'RP']


sekarang tinggal masalah null untuk EDA

**e. handling missing values**

In [11]:
# Check NA dan Percentage
datasets.isna().sum()
percentage_na = datasets.isna().sum()/len(datasets)
print(percentage_na*100)

Name                0.011815
Platform            0.000000
Year_of_Release     0.000000
Genre               0.011815
Publisher           0.324905
NA_Sales            0.000000
EU_Sales            0.000000
JP_Sales            0.000000
Other_Sales         0.000000
Global_Sales        0.000000
Critic_Score       51.205104
Critic_Count       51.205104
User_Score         39.987004
User_Count          0.000000
Developer          39.508507
Rating             40.382798
dtype: float64


so our data has some missing values here,
- Name
so for name im just gonna drop it because we cannot change it, cause we dont have any informationa about the game that is null
- Genre 
for genre i'm gonna a drop it as well because the name also nan so we dont have info to input it.
- Publisher
for publisher it's kinda low so we're just gonna drop it like name cause we dont have any info to input it.
- Critic Score and Critic Count, 
both of these we're gonna replace it with mean or median why ? although we see it has 51% of the missing values in our data almost half of it. BUT both of these very important column so we need to hold it and not drop it so yeah we're gonna input it.
- User score
as same as Critic Score and count, we'll also gonna input these cause it's very useful for our system later to give recomendation. This one is tricky cause it's nan is not a nan type so we'll gonna change it first.
- Developer
for developper because it's still below 50% we're gonna just input it with threshold because we dont have info who's the developper is and cause we cannot risk losing more of the data that can help the system to recomend the game later
- Rating
as same as developper we're gonna give it the mode to input the ratings.

In [12]:
datasets['Critic_Score'] = pd.to_numeric(datasets['Critic_Score'], errors='coerce')
datasets['Critic_Score'] = datasets['Critic_Score'].fillna(datasets['Critic_Score'].mean())
datasets['Critic_Count'] = pd.to_numeric(datasets['Critic_Count'], errors='coerce').fillna(datasets['Critic_Count'].median())
datasets['User_Score'] = datasets['User_Score'].replace('nan', np.nan)
datasets['User_Score'] = pd.to_numeric(datasets['User_Score'], errors='coerce')
datasets['User_Score'] = datasets['User_Score'].fillna(datasets['User_Score'].median())
datasets['Developer'] = datasets['Developer'].fillna('Unknown')
datasets['Rating'] = datasets['Rating'].fillna(datasets['Rating'].mode()[0])
datasets = datasets.dropna()

In [13]:
datasets.isna().sum()
percentage_na = datasets.isna().sum()/len(datasets)
print(percentage_na*100)

Name               0.0
Platform           0.0
Year_of_Release    0.0
Genre              0.0
Publisher          0.0
NA_Sales           0.0
EU_Sales           0.0
JP_Sales           0.0
Other_Sales        0.0
Global_Sales       0.0
Critic_Score       0.0
Critic_Count       0.0
User_Score         0.0
User_Count         0.0
Developer          0.0
Rating             0.0
dtype: float64


# Content Based Learning (Features : Genre , Platform, Publisher, Rating )

because the goals of these problem is a content based learning so we are using categorical features such as genre platform publisher and rating

In [35]:
datasets['Combined_Features'] = datasets['Genre'] + " " + datasets['Platform'] + " " + datasets['Publisher'] + " " + datasets['Rating'].astype(str)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(datasets['Combined_Features'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
def get_recommendations(name, cosine_sim=cosine_sim, data=datasets, top_n=5):
    idx = data[data['Name'].str.contains(name, case=False, na=False)].index
    if len(idx) == 0:
        return f"No match found for '{name}'."
    idx = idx[0]  
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_indices = [i[0] for i in sim_scores[1:top_n + 1]] 

    return data.iloc[sim_indices][['Name', 'Platform', 'Year_of_Release', 'Critic_Score', 'User_Score', 'Critic_Count', 'User_Count', 'Rating', 'Combined_Features']]

/var/folders/k8/0jttqwyj0vdg5y3q78vqjy1c0000gn/T/ipykernel_9569/3069712586.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets['Combined_Features'] = datasets['Genre'] + " " + datasets['Platform'] + " " + datasets['Publisher'] + " " + datasets['Rating'].astype(str)


In [34]:
recommendation_mario = get_recommendations("Mario Kart")
recommendation_mario

,Name,Platform,Year_of_Release,Critic_Score,User_Score,Critic_Count,User_Count,Rating,Combined_Features
3237,Donkey Kong Barrel Blast,Wii,2007,46.0,6.8,34.0,26.0,E,Racing Wii Nintendo E
3782,Excite Truck,Wii,2006,72.0,8.4,57.0,83.0,E,Racing Wii Nintendo E
6528,ExciteBots: Trick Racing,Wii,2009,77.0,8.6,29.0,26.0,E,Racing Wii Nintendo E
16832,Donkey Kong Barrel Blast,Wii,2007,46.0,6.8,34.0,26.0,E,Racing Wii Nintendo E
107,The Legend of Zelda: Twilight Princess,Wii,2006,95.0,9.0,73.0,1658.0,T,Action Wii Nintendo T


In [33]:
recommendations_zelda = get_recommendations("The Legend of Zelda: Twilight Princess")
recommendations_zelda

,Name,Platform,Year_of_Release,Critic_Score,User_Score,Critic_Count,User_Count,Rating,Combined_Features
1449,Metroid: Other M,Wii,2010,79.000000,6.5,71.0,482.0,T,Action Wii Nintendo T
2723,PokePark 2: Wonders Beyond,Wii,2011,60.000000,7.0,18.0,27.0,E,Action Wii Nintendo E
4183,FlingSmash,Wii,2010,48.000000,6.8,26.0,13.0,E,Action Wii Nintendo E
4441,Samurai Warriors 3,Wii,2009,55.000000,7.2,26.0,22.0,T,Action Wii Nintendo T
10096,Zangeki no Reginleiv,Wii,2010,69.037893,7.5,22.0,-661.0,E,Action Wii Nintendo E


In [37]:
recommendations_excitebots = get_recommendations("Yakuza: Dead Souls")
recommendations_excitebots

,Name,Platform,Year_of_Release,Critic_Score,User_Score,Critic_Count,User_Count,Rating,Combined_Features
2890,Dynasty Warriors 6,PS3,2007,59.0,7.3,31.0,49.0,T,Action PS3 Tecmo Koei T
3337,Dynasty Warriors 8,PS3,2013,68.0,8.5,26.0,75.0,T,Action PS3 Tecmo Koei T
3703,Dynasty Warriors: Gundam 3,PS3,2010,58.0,7.5,18.0,20.0,T,Action PS3 Tecmo Koei T
4136,Dynasty Warriors Gundam,PS3,2007,60.0,6.9,29.0,16.0,T,Action PS3 Tecmo Koei T
4471,Ninja Gaiden 3,PS3,2012,58.0,5.0,43.0,152.0,M,Action PS3 Tecmo Koei M


# FINAL CONCLUSION

1. For "Mario Kart":
- Genre: Racing
- Platform: Nintendo Wii
- Publisher: Nintendo
- Rating: E
if we compare with the result that have been given we can see 4 on the top has matching answer with mario kart. The last one legend of zelda is diffent on Rating and Genre, but it's the same for the other Categorical. So from this we can conclude that the recomender system gave the closest combined features with marip kart.


2. For "The Legend of Zelda: Twilight Princess":
- Genre: Action/Adventure
- Platform: Nintendo Wii
- Publisher: Nintendo
- Rating: T
for legend zelda as well we can see the combined feature is super duper close to legend zelda which 3 data only differs on the Rating. Here we can see for legend zelda a lot of the recomendattion is focus on action wii and nintendo like Metroid Other : M , Pokepark 2, Fling Smash, Samurai Warrior 3, Zangeki No Reginieiv. Although some of the column has different Rating, But we can see it's very associated with the game legend of zelda.

3.  For "Yakuza: Dead Souls":
- Genre: Actiom
- Platform: PS3
- Publisher: Tecmo Koel
- Rating: T
For yakuza dead souls, we can also see that the recomendation is very asscoiated, for the genre 5/5 are on the same genre with yakuza dead souls, next for the platform is also the same 5/5 followed up by 5/5 publisher and closed with 4/5 Rating. Although the last rating is different but we can see that yakuza dead souls and the other games such as dynasty warriors and ninja Gaiden has a lot in common.

# KESIMPULAN
there are a lot of dirty numerical column in the datasets, so we gotta clean it according to the 5 , 5-50 and above 50% rules. But there's a good thing about this dirty data that is 50% of the missing values is focus on the numerical data. WHY? because for recomendatiion we are using based content learning, which focuses on the categorical side, Although we also need to clean the data because we need to do full EDA from the goals of this problem. For the recomendation System we can see that it's working very good because it's given asscoiated category as the string that we inputted. So in the end we can said that our system has a good way to give a recomendation for us.